In [3]:
import numpy as np
import pandas as pd

In [2]:
def save_dataframe_to_csv(dataframe, filename):
    df = pd.DataFrame(dataframe)
    columns_to_exclude_index = [0, 1, 7, 8, 10, 11, 13, 14, 16, 17, 19, 20, 26, 34]  
    columns_to_multiply_by_minus_1_index = [col_idx for col_idx in range(len(df.columns)) if col_idx not in columns_to_exclude_index]
    df.iloc[:, columns_to_multiply_by_minus_1_index] = df.iloc[:, columns_to_multiply_by_minus_1_index] * -1
    df.to_csv(filename, index=False)

In [4]:
data1 = pd.read_csv("Dataset/win/Silver_I.csv")
data2 = pd.read_csv("Dataset/win/Silver_II.csv")
data3 = pd.read_csv("Dataset/win/Silver_III.csv")
data4 = pd.read_csv("Dataset/win/Silver_IV.csv")

In [5]:
windata = pd.concat([data1,data2,data3,data4])

In [17]:
save_dataframe_to_csv(windata,"Dataset/lose/Gold_lose_concats.csv")

In [6]:
data5 = pd.read_csv("Dataset/lose/Silver_I_lose.csv")
data6 = pd.read_csv("Dataset/lose/Silver_II_lose.csv")
data7 = pd.read_csv("Dataset/lose/Silver_III_lose.csv")
data8 = pd.read_csv("Dataset/lose/Silver_IV_lose.csv")

In [29]:
losedata = pd.read_csv("Dataset/lose/Silver_lose_concat.csv")

In [8]:
losedata = pd.concat([data5,data6,data7,data8])

In [9]:

data = pd.concat([windata, losedata])

In [15]:
print(data['result'])

0       1
1       1
2       1
3       1
4       1
       ..
2714   -1
2715   -1
2716   -1
2717   -1
2718   -1
Name: result, Length: 22664, dtype: int64


In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# df = pd.read_csv('Grandmaster_concat.csv')


data['result'] = data['result'].replace({-1: 0})


features = ['Diff_LV', 'Diff_CS', 'Diff_jglCS', 'Diff-K', 'Diff-K-top', 'Diff-K-jug', 'Diff-K-mid', 'Diff-K-ad',
            'Diff-K-sup', 'invadeKill', 'Diff-A', 'Diff_WARDplaced', 'Diff-ControlWARDplaced', 'Diff_WARDkill',
            'Diff_Inhibitor', 'Diff_TOWERkill', 'Diff_FirstDRAGON', 'Diff_FirstHERALD', 'Diff_Firsttower', 'Diff_FirstBLOOD']


X_train, X_test, y_train, y_test = train_test_split(data[features], data['result'], test_size=0.25, random_state=32)
#df[features]을 통해 특정 칼럼들로 학습 데이터로 사용한다. 그리고 df['result'] 이 부분을 통해 각 게임에 대한 승패 결과를 교육(?)
model = XGBClassifier(n_estimators=500, learning_rate=0.2, max_depth=12, random_state=32)#n_estimators는 의사 결정 트리의 수. 값을 높게 설정 할 수록 정확도가 좋다고 한다.
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("정확도:", accuracy)

정확도: 0.7911603888213852


In [28]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

data['result'] = data['result'].replace({-1: 0})


features = ['Diff_LV', 'Diff_CS', 'Diff_jglCS', 'Diff-K', 'Diff-K-top', 'Diff-K-jug', 'Diff-K-mid', 'Diff-K-ad',
            'Diff-K-sup', 'invadeKill', 'Diff-A', 'Diff_WARDplaced', 'Diff-ControlWARDplaced', 'Diff_WARDkill',
            'Diff_Inhibitor', 'Diff_TOWERkill', 'Diff_FirstDRAGON', 'Diff_FirstHERALD', 'Diff_Firsttower', 'Diff_FirstBLOOD']


X_train, X_test, y_train, y_test = train_test_split(data[features], data['result'], test_size=0.25, random_state=32)
model = RandomForestClassifier(n_estimators=120, random_state=32)#n_estimators는 의사 결정 트리의 수. 값을 높게 설정 할 수록 정확도가 좋다고 한다.
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("정확도:", accuracy)

정확도: 0.7998177399756987


In [29]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

data['result'] = data['result'].replace({-1: 0})


features = ['Diff_LV', 'Diff_CS', 'Diff_jglCS', 'Diff-K', 'Diff-K-top', 'Diff-K-jug', 'Diff-K-mid', 'Diff-K-ad',
            'Diff-K-sup', 'invadeKill', 'Diff-A', 'Diff_WARDplaced', 'Diff-ControlWARDplaced', 'Diff_WARDkill',
            'Diff_Inhibitor', 'Diff_TOWERkill', 'Diff_FirstDRAGON', 'Diff_FirstHERALD', 'Diff_Firsttower', 'Diff_FirstBLOOD']


X_train, X_test, y_train, y_test = train_test_split(data[features], data['result'], test_size=0.25, random_state=32)
model = LGBMClassifier(n_estimators=400)#n_estimators는 의사 결정 트리의 수. 값을 높게 설정 할 수록 정확도가 좋다고 한다.
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("정확도:", accuracy)

[LightGBM] [Info] Number of positive: 9888, number of negative: 9862
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001223 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1085
[LightGBM] [Info] Number of data points in the train set: 19750, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500658 -> initscore=0.002633
[LightGBM] [Info] Start training from score 0.002633
정확도: 0.7777946537059538


In [30]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

data['result'] = data['result'].replace({-1: 0})


features = ['Diff_LV', 'Diff_CS', 'Diff_jglCS', 'Diff-K', 'Diff-K-top', 'Diff-K-jug', 'Diff-K-mid', 'Diff-K-ad',
            'Diff-K-sup', 'invadeKill', 'Diff-A', 'Diff_WARDplaced', 'Diff-ControlWARDplaced', 'Diff_WARDkill',
            'Diff_Inhibitor', 'Diff_TOWERkill', 'Diff_FirstDRAGON', 'Diff_FirstHERALD', 'Diff_Firsttower', 'Diff_FirstBLOOD']


X_train, X_test, y_train, y_test = train_test_split(data[features], data['result'], test_size=0.25, random_state=32)
model = CatBoostClassifier(iterations=100, random_state=123)#n_estimators는 의사 결정 트리의 수. 값을 높게 설정 할 수록 정확도가 좋다고 한다.
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("정확도:", accuracy)

Learning rate set to 0.304187
0:	learn: 0.5908507	total: 150ms	remaining: 14.8s
1:	learn: 0.5376222	total: 162ms	remaining: 7.92s
2:	learn: 0.5094473	total: 173ms	remaining: 5.58s
3:	learn: 0.4970889	total: 186ms	remaining: 4.45s
4:	learn: 0.4881441	total: 200ms	remaining: 3.8s
5:	learn: 0.4821653	total: 211ms	remaining: 3.31s
6:	learn: 0.4783542	total: 223ms	remaining: 2.96s
7:	learn: 0.4758225	total: 237ms	remaining: 2.73s
8:	learn: 0.4744318	total: 248ms	remaining: 2.5s
9:	learn: 0.4731509	total: 258ms	remaining: 2.32s
10:	learn: 0.4713946	total: 272ms	remaining: 2.2s
11:	learn: 0.4705387	total: 286ms	remaining: 2.1s
12:	learn: 0.4693619	total: 298ms	remaining: 1.99s
13:	learn: 0.4682308	total: 311ms	remaining: 1.91s
14:	learn: 0.4671658	total: 321ms	remaining: 1.82s
15:	learn: 0.4658264	total: 332ms	remaining: 1.74s
16:	learn: 0.4644178	total: 342ms	remaining: 1.67s
17:	learn: 0.4632631	total: 353ms	remaining: 1.61s
18:	learn: 0.4620082	total: 364ms	remaining: 1.55s
19:	learn: 0.46

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score

data['result'] = data['result'].replace({-1: 0})


features = ['Diff_LV', 'Diff_CS', 'Diff_jglCS', 'Diff-K', 'Diff-K-top', 'Diff-K-jug', 'Diff-K-mid', 'Diff-K-ad',
            'Diff-K-sup', 'invadeKill', 'Diff-A', 'Diff_WARDplaced', 'Diff-ControlWARDplaced', 'Diff_WARDkill',
            'Diff_Inhibitor', 'Diff_TOWERkill', 'Diff_FirstDRAGON', 'Diff_FirstHERALD', 'Diff_Firsttower', 'Diff_FirstBLOOD']


X_train, X_test, y_train, y_test = train_test_split(data[features], data['result'], test_size=0.25, random_state=32)
model = ExtraTreesClassifier(n_estimators=100, random_state=32)#n_estimators는 의사 결정 트리의 수. 값을 높게 설정 할 수록 정확도가 좋다고 한다.
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("정확도:", accuracy)

정확도: 0.7952612393681653
